## AUTOMAÇÃO DA SOLICITAÇÃO DO VÍNCULO DE RENOVAÇÃO

In [0]:
### Instalação de bibliotecas
##---------------------------

%pip install jira python-gitlab

In [0]:
# Bibliotecas
import mysql.connector
import pandas as pd
import gitlab
import time
from jira import JIRA
from datetime import datetime
from AVITA.automacoes import funcoes as f

### DataFrames

In [0]:
### Configura conexão com o banco de dados do Financeiro: db-financeiro
#-----------------------------------------------------------------------------
jdbcUserFinanceiro = dbutils.secrets.get(scope = personal_scope, key = personal_key)
jdbcPasswordFinanceiro = dbutils.secrets.get(scope = personal_scope, key = personal_key)
jdbcHostnameFinanceiro = dbutils.secrets.get(scope = personal_scope, key = personal_key)
jdbcDatabaseFinanceiro = dbutils.secrets.get(scope = personal_scope, key = personal_key)
jdbcPort = personal_port
tabela_pre_prod = "table_name"
tabela_processados = "table_name"

jdbcUrlFinanceiro = "jdbc:mysql://{0}:{1}/{2}".format(jdbcHostnameFinanceiro, jdbcPort, jdbcDatabaseFinanceiro)

connectionPropertiesFinanceiro = {
  "user" : jdbcUserFinanceiro,
  "password" : jdbcPasswordFinanceiro,
  "driver" : "com.mysql.jdbc.Driver"
}

# Informações de conexão
config = {
    'user': jdbcUserFinanceiro,
    'password': jdbcPasswordFinanceiro,
    'host': jdbcHostnameFinanceiro,
    'database': jdbcDatabaseFinanceiro,
    'port': jdbcPort
}

# Conecta ao banco de dados
connection = mysql.connector.connect(**config)

# Executa uma consulta SQL e obtém os resultados como um DataFrame do Pandas
query_pre_prod = f"SELECT * FROM {tabela_pre_prod}"
df_pre_prod = pd.read_sql(query_pre_prod, con=connection)

query_processados = f"SELECT * FROM {tabela_processados}"
df_processados = pd.read_sql(query_processados, con=connection)

# Conexão não é fechada pois será utilizada para inserir dados

In [0]:
### Configura conexão com o banco de dados ReportDB
#-----------------------------------------------------------------------------

# Configurações
host = dbutils.secrets.get(scope = personal_scope, key = personal_key)
port = personal_port
user = dbutils.secrets.get(scope = personal_scope, key = personal_key)
password = dbutils.secrets.get(scope = personal_scope, key = personal_key)
database = dbutils.secrets.get(scope = personal_scope, key = personal_key)
tabela_op_policy = 'table_name'
tabela_renew = 'table_name'
tabela_core_user = 'table_name'

# Informações de conexão
config = {
    'user': user,
    'password': password,
    'host': host,
    'database': database,
    'port': port
}

# Criar a conexão com o banco de dados MySQL
connection = mysql.connector.connect(**config)

# Executar a consulta SQL para obter os dados da tabela
query_op_policy = f"SELECT * FROM {tabela_op_policy}"
df_op_policy = pd.read_sql(query_op_policy, con=connection)

query_op_policy_renew = f"SELECT * FROM {tabela_renew}"
df_op_policy_renew = pd.read_sql(query_op_policy_renew, con=connection)

query_core_user = f"SELECT * FROM {tabela_core_user}"
df_core_user = pd.read_sql(query_core_user, con=connection)

query_op_policy_tratado = f""" 
select *
from {tabela_op_policy}
where true 
AND (cláusulas específicas da tabela em questão para tratamento de possíveis apólices com erro)"""
df_op_policy_tratado = pd.read_sql(query_op_policy_tratado, con=connection)

# Fecha a conexão
connection.close()

### Processo de verificação e validação

In [0]:
# Atribuindo ID's dos dataframes em listas para o processo de verificação

ids_pre_prod = df_pre_prod['id'].to_list()
ids_processados = df_processados['id_pre_prod'].to_list()
ids_operational_policy = df_op_policy_tratado['id'].to_list()
ids_operational_policy_completa = df_op_policy['id'].to_list()
ids_old_pre_prod = df_pre_prod['id_policy_old'].to_list()
ids_new_pre_prod = df_pre_prod['id_policy_new'].to_list()
ids_old_renew = df_op_policy_renew['id_policy_old'].to_list()
ids_new_renew = df_op_policy_renew['id_policy_new'].to_list()
status_renew_aceitaveis = [1, 2, 3, 5] 
DePara_status_renew = {
  1 : "Notificação enviada",
  2 : "Em cancelamento",
  3 : "Em renovação",
  4 : "Renovação feita",
  5 : "Cancelada"
}

In [0]:
# Lista de ID's pre_prod para fazer o vínculo
IDs_vincular = []
renegados = []

# Listas com logs de erro
erro_0 = [] # id_new consta endossado, mas tem correlation diferente do id_old
erro_1 = [] # id_new tem renovação vigente como id_new
erro_2 = [] # id_old tem renovação vigente como id_old
erro_3 = [] # id_old não pode ser renovado por ter status diferente de: Ativo, expirado ou endossado
erro_4 = [] # id_new não pode ser renovado por ter status diferente de: Ativo, expirado ou endossado
erro_5 = [] # id_old é um endosso de cancelamento
erro_6 = [] # id_new é um endosso de cancelamento
erro_7 = [] # id_old não possui número de apólice
erro_8 = [] # id_new não possui número de apólice
erro_9 = [] # id_old não possui data de emissão
erro_10 = [] # id_new não possui data de emissão
erro_11 = [] # id_old não encontrado
erro_12 = [] # id_new não encontrado
erro_13 = [] # id_old cancelado logicamente
erro_14 = [] # id_new cancelado logicamente

In [0]:
### Passo a Passo processo de verificação: Validar se o vínculo pode ser feito

# 1. Verifica se o id na tabela pre_prod já existe na tabela de processados.
  # Se já existe, ignora, pois é uma solicitação antiga;
  # Se não existe, então é uma solicitação nova;

# 2. Verifica se o ID new e old existem no DataFrame operational_policy_tratado
  # OBS: Operational_policy_tratado é a tabela operational_policy com os seguintes tratamentos: 
    # ID's sem logical delete;
    # Com data de emissão;
    # Com número de apólice;
    # Dentro dos status: ativo, expirado e endossado;
    # Que não seja endosso de cancelamento;

  # 3. Se estiver, então verifica se o ID old já está na tabela operational_policy_renew com renovação vigente como ID old, isto é:
    # Se está na operational_policy_renew com status = 4 e sem data de delete lógico;
    # Se sim, inclui esse ID na lista de solicitações com erro;
     
  # 4. Se não, verifica se o ID new já tem renovação vigente como ID new;
    # Se sim, inclui esse ID na lista de solicitações com erro;

  # 5. Se não, verifica se o ID new é um endosso (type = 43)
    # Se não, inclui na lista de solicitações a serem feitas;
    # 6. Se sim, verifica se o ID new e old estão no mesmo correlation;
      # Se não, inclui esse ID na lista de solicitações com erro;
      # Se sim, inclui na lista de solicitações a serem feitas;
  
# 7. Se alugm dos ID's (new ou old) não for encontrado no DataFrame operational_policy_tratado, o BOT verifica qual o erro e inclui na lista de solicitações com erro:
  # ID new ou old não existe no banco;
  # ID new ou old com status diferente de: Ativo, expirado ou endossado;
  # ID new ou old é endosso de cancelamento;
  # ID new ou old não tem número de apólice;
  # ID new ou old não tem data de emissão;
  # ID new ou old está deletado logicamente.

In [0]:
### Verificação para validar se de fato o vínculo pode ser realizado

for i, id in enumerate(ids_pre_prod):
  # 1. Verifica se o id na tabela pre_prod já existe na tabela de processados
  if id not in ids_processados:
    # 2. Verifica se o ID new e old existem no DataFrame operational_policy_tratado
    if ids_old_pre_prod[i] in ids_operational_policy and ids_new_pre_prod[i] in ids_operational_policy:
      # 3. Se estiver, então verifica se o ID old já está na tabela operational_policy_renew com renovação vigente
      if ids_old_pre_prod[i] not in ids_old_renew or df_op_policy_renew.loc[df_op_policy_renew['id_policy_old'] == int(ids_old_pre_prod[i]), "status"].iloc[0] in status_renew_aceitaveis or not df_op_policy_renew.loc[df_op_policy_renew['id_policy_old'] == int(ids_old_pre_prod[i]), "dt_logical_delete"].isna().all():
        # 4. Se não, verifica se o ID new já tem renovação vigente
        if not ids_new_pre_prod[i] in ids_new_renew or df_op_policy_renew.loc[df_op_policy_renew['id_policy_new'] == int(ids_new_pre_prod[i]), "status"].iloc[0] in status_renew_aceitaveis or not df_op_policy_renew.loc[df_op_policy_renew['id_policy_new'] == int(ids_new_pre_prod[i]), "dt_logical_delete"].isna().all():
          # 5. Se não, verifica se o ID new é um endosso
          if df_op_policy_tratado.loc[df_op_policy_tratado['id'] == ids_new_pre_prod[i], 'id_enum_type_solicitation'].iloc[0] == 43:
            # 6. Se sim, verifica se o ID new e old estão no mesmo correlation;
            if df_op_policy_tratado.loc[df_op_policy_tratado['id'] == ids_new_pre_prod[i], 'txt_correlation_id'].iloc[0] == df_op_policy_tratado.loc[df_op_policy_tratado['id'] == ids_old_pre_prod[i], 'txt_correlation_id'].iloc[0]:
              IDs_vincular.append(id)
            else:
              erro_0.append(id)
          else:
            IDs_vincular.append(id)
        else:
          erro_1.append(id)
      else:
        erro_2.append(id)


    # 7. Se alugm dos ID's (new ou old) não for encontrado no DataFrame operational_policy_tratado, o BOT verifica qual o erro

    # ID new ou old não existe no banco
    elif not ids_old_pre_prod[i] in ids_operational_policy_completa:
      erro_11.append(id)
    elif not ids_new_pre_prod[i] in ids_operational_policy_completa:
      erro_12.append(id)
    
    # ID new ou old com status diferente de: Ativo, expirado ou endossado
    elif not df_op_policy.loc[df_op_policy['id'] == int(ids_old_pre_prod[i]), 'Status'].iloc[0] in [6, 8, 17]:
      erro_3.append(id)
    elif not df_op_policy.loc[df_op_policy['id'] == int(ids_new_pre_prod[i]), 'Status'].iloc[0] in [6, 8, 17]:
      erro_4.append(id)

    # ID new ou old é endosso de cancelamento
    elif df_op_policy.loc[df_op_policy['id'] == int(ids_old_pre_prod[i]), 'enum_endorsement_type'].iloc[0] == 1:
      erro_5.append(id)
    elif df_op_policy.loc[df_op_policy['id'] == int(ids_new_pre_prod[i]), 'enum_endorsement_type'].iloc[0] == 1:
      erro_6.append(id)

    # ID new ou old não tem número de apólice
    elif df_op_policy.loc[df_op_policy['id'] == int(ids_old_pre_prod[i]), 'txt_policy_number'].isna().all():
      erro_7.append(id)
    elif df_op_policy.loc[df_op_policy['id'] == int(ids_new_pre_prod[i]), 'txt_policy_number'].isna().all():
      erro_8.append(id)

    # ID new ou old não tem data de emissão
    elif df_op_policy.loc[df_op_policy['id'] == int(ids_old_pre_prod[i]), 'dt_emission'].isna().all():
      erro_9.append(id)
    elif df_op_policy.loc[df_op_policy['id'] == int(ids_new_pre_prod[i]), 'dt_emission'].isna().all():
      erro_10.append(id)

    # ID new ou old está deletado logicamente
    elif not pd.isnull(df_op_policy.loc[df_op_policy['id'] == int(ids_old_pre_prod[i]), 'dt_logical_delete'].iloc[0]):
      erro_13.append(id)
    elif not pd.isnull(df_op_policy.loc[df_op_policy['id'] == int(ids_new_pre_prod[i]), 'dt_logical_delete'].iloc[0]):
      erro_14.append(id)

    # No caso de passar por todas as verificações, é incluído nessa lista para entendermos o motivo
    else:
      renegados.append(id)

# OBS: Adicionamos á lista de logs de erros e ao ID's_vincular o id pre_prod, e não o id_new ou id_old

### Logs de erro

In [0]:
# Colocando as listas com erros dentro do dicionário de log de erros:
logs_erros = {}

logs_erros[0] = erro_0, ["ID da renovação consta endossado, mas não está na fila de endossos do ID renovado"] 
logs_erros[1] = erro_1, ["ID da renovação tem renovação vigente sendo renovação"] 
logs_erros[2] = erro_2, ["ID renovado tem renovação vigente sendo o renovado"] 
logs_erros[3] = erro_3, ["ID renovado não pode ser renovado por ter status diferente de: Ativo, expirado ou endossado"] 
logs_erros[4] = erro_4, ["ID da renovação não pode renovar por ter status diferente de: Ativo, expirado ou endossado"] 
logs_erros[5] = erro_5, ["ID renovado é um endosso de cancelamento"] 
logs_erros[6] = erro_6, ["ID da renovação é um endosso de cancelamento"] 
logs_erros[7] = erro_7, ["ID renovado não possui número de apólice"] 
logs_erros[8] = erro_8, ["ID da renovação não possui número de apólice"] 
logs_erros[9] = erro_9, ["ID renovado não possui data de emissão"] 
logs_erros[10] = erro_10, ["ID da renovação não possui data de emissão"] 
logs_erros[11] = erro_11, ["ID renovado não encontrado"] 
logs_erros[12] = erro_12, ["ID da renovação não encontrado"] 
logs_erros[13] = erro_13, ["ID renovado cancelado logicamente"]
logs_erros[14] = erro_14, ["ID da renovação cancelado logicamente"]

In [0]:
# Inserir dados na tabela de logs com erro
tabela = "table_name"

for ids, mensagem_erro in logs_erros.values():
  for id in ids:
    if id:
      id_policy_new = int(df_pre_prod.loc[df_pre_prod['id'] == id, 'id_policy_new'].iloc[0])
      id_policy_old = int(df_pre_prod.loc[df_pre_prod['id'] == id, 'id_policy_old'].iloc[0])
      dados = {"id" : id, 
              "id_new" : id_policy_new, 
              "id_old" : id_policy_old, 
              "message_error" : str(mensagem_erro[0])}
      sql_inserir_dados = f"INSERT INTO {tabela} ({', '.join(dados.keys())}) VALUES ({', '.join(['%s']*len(dados))})"
      cursor.execute(sql_inserir_dados, tuple(dados.values()))
      connection.commit()

### Integração com o Jira

In [0]:
### Configura conexão com o Jira
#-----------------------------------------------------------------------------


# Construct a Client Instance
jiraOptions = {'server': "https://example.atlassian.net"}
email_id = "bot_email"
token = "personal_token"

# Pass the Authentication Parameter
jira = JIRA(options=jiraOptions, basic_auth=(email_id, token))

In [0]:
### Passo a Passo para criação de cards no Jira com logs de erros:

# 1. Criar chamado
# 2. Atribuir para mim
# 3. Atribuir o usuário como relator
# 4. Passa para a fila de suporte
# 5. Cancelar chamado com mensagem de erro (email automático)
  # Campos obrigatórios: Motivo do cancelamento (aberto), Causa Raiz (Solicitação Interna), Comentar (aberto)

In [0]:
# Card no Jira para Logs com erros
chamados_erros = {}

# Loop for para percorrer o dicionário com erros
for ids, mensagem_erro in logs_erros.values():
  for id in ids:
    # 1. Criar uma nova issue
    id_policy_new = int(df_pre_prod.loc[df_pre_prod['id'] == id, 'id_policy_new'].iloc[0])
    id_policy_old = int(df_pre_prod.loc[df_pre_prod['id'] == id, 'id_policy_old'].iloc[0])
    email_usuario = df_pre_prod.loc[df_pre_prod['id'] == id, 'email_solicitante'].iloc[0]
    usuario = df_core_user.loc[df_core_user['txt_email'] == email_usuario, 'txt_name'].iloc[0]
    nova_issue = {
      'project': {'key': 'ABCD'},  # ABCD : Key do projeto
      'summary': f'Vínculo de renovação ID\'s: {id_policy_old} e {id_policy_new}',
      'description': f"""Solicitação vínculo de renovação: favor vincular ID da renovação {id_policy_new} ao ID renovado {id_policy_old}.
      Solicitado por {usuario}.
      Avaliado pelo BOT: Vínculo de renovação.""",
      'issuetype': {'name': 'Type'}  # Task : Type
    }

    issue = jira.create_issue(fields=nova_issue)

    time.sleep(1)

    # 2. Atribuir a issue para mim
    responsavel_nome = 'Username'
    jira.assign_issue(issue, responsavel_nome)


    # 3. Atribuindo o usuário como relator
    issue_key = issue.key # Chave do chamado (issue key)
    novo_relator_email = email_usuario # E-mail do novo relator
      # Obter informações do usuário usando o e-mail
    novo_relator = jira.search_users(query=novo_relator_email)
    if novo_relator:
        novo_relator_id = novo_relator[0].accountId
        # Atualizar o chamado para alterar o relator
        jira.issue(issue_key).update(fields={'reporter': {'id': novo_relator_id}})
    else:
        print(f'Usuário com e-mail {novo_relator_email} não encontrado.')

    time.sleep(1)

    # 4. Transição para fila de suporte
    id_transicao_suporte = 000 # Example
    campos_customizados = { 
        'customfield_XXXXX': {'id': 'XXX'}, # Squad 
        'customfield_XXXXX': {'id': 'XXX'}, # Fluxo 
        'customfield_XXXXX': {'id': 'XXX'} # Ação do usuário 
    }

    transicao_args = {
      'transition': id_transicao_suporte,
      'fields': campos_customizados
    }

    jira.transition_issue(issue, **transicao_args)

    time.sleep(1)

    # 5. Cancelar chamado
    id_cancelamento = 000
    comentario_cancelamento = f"""Olá {novo_relator_email}, Tudo bem?. 
    Nossa automação identificou que o vínculo não pôde ser realizado. 
    O ID da renovação {id_policy_new} não pôde ser vinculado ao ID renovado {id_policy_old} segundo o erro: 
    {mensagem_erro[0]}"""

    comment = [{'add': {'body': comentario_cancelamento}}]

    transicao_args = {
        'transition': id_cancelamento,
        'comment': comentario_cancelamento
    }

    jira.transition_issue(issue, **transicao_args)

    # Chamado adicionado ao dicionário
    chamados_erros[id] = issue.key

for chamado in chamados_erros.values():
  print(f'Chamado {chamado} criado e cancelado!')

In [0]:
# Inserir id_pre_prod na tabela de processados com Status = 3 (erro)
tabela = "table_name"

for id in chamados_erros.keys():
  if id:
    chave_chamado = chamados_erros[id]
    dados = {"id" : id, 
            "status" : 3, # Status 3 = erro identificado 
            "chave_chamado" : chave_chamado} 
    sql_inserir_dados = f"INSERT INTO {tabela} ({', '.join(dados.keys())}) VALUES ({', '.join(['%s']*len(dados))})"
    cursor.execute(sql_inserir_dados, tuple(dados.values()))
    connection.commit()

In [0]:
### Passo a Passo para criação de cards no Jira com vínculos a serem feitos

# 1. Criar chamado
# 2. Atribuir para mim
# 3. Atribuir o usuário como relator
# 4. Passar para "Fila de suporte" e depois passar para "Em andamento"
  # Preencher campos obrigatórios: fluxo, ação do usuário e squad
# 5. Registrar mensagem: "Vincular id_new ao id_old no operational_policy_renew. Nossa automação já está trabalhando nisso"
# 6. Após merge ser criado, passar chamado para "Aguardando deploy"
# 7. Após merge ser realizado, concluir chamado

In [0]:
# Card no Jira para vínculos a serem feitos
chamados = {}

# Loop for para percorrer a lista com os ID's a serem vinculado
for id in IDs_vincular:
  # 1. Criar uma nova issue
  id_policy_new = int(df_pre_prod.loc[df_pre_prod['id'] == id, 'id_policy_new'].iloc[0])
  id_policy_old = int(df_pre_prod.loc[df_pre_prod['id'] == id, 'id_policy_old'].iloc[0])
  email_usuario = df_pre_prod.loc[df_pre_prod['id'] == id, 'email_solicitante'].iloc[0]
  usuario = df_core_user.loc[df_core_user['txt_email'] == email_usuario, 'txt_name'].iloc[0]
  nova_issue = {
    'project': {'key': 'ABCD'},  # ABCD : Key do projeto
    'summary': f'Vínculo de renovação ID\'s: {id_policy_old} e {id_policy_new}',
    'description': f"""Solicitação vínculo de renovação: favor vincular ID da renovação {id_policy_new} ao ID renovado {id_policy_old}.
    Solicitado por {usuario}.
    Avaliado pelo BOT: Vínculo de renovação.""",
    'issuetype': {'name': 'Type'}  # Task : Type
    }
  }

  issue = jira.create_issue(fields=nova_issue)

  time.sleep(1)

  # 2. Atribuir a issue para mim
  responsavel_nome = 'Username'
  jira.assign_issue(issue, responsavel_nome)


  # 3. Atribuindo o usuário como relator
  issue_key = issue.key # Chave do chamado (issue key)
  novo_relator_email = email_usuario # E-mail do novo relator
    # Obter informações do usuário usando o e-mail
  novo_relator = jira.search_users(query=novo_relator_email)
  if novo_relator:
      novo_relator_id = novo_relator[0].accountId
      # Atualizar o chamado para alterar o relator
      jira.issue(issue_key).update(fields={'reporter': {'id': novo_relator_id}})
  else:
      print(f'Usuário com e-mail {novo_relator_email} não encontrado.')

  time.sleep(1)

  # 4. Transição para fila de suporte
  id_transicao_suporte = 000
  campos_customizados = { 
      'customfield_XXXXX': {'id': 'XXXXX'}, # Squad 
      'customfield_XXXXX': {'id': 'XXXXX'}, # Fluxo 
      'customfield_XXXXX': {'id': 'XXXXX'} # Ação do usuário 
  }

  transicao_args = {
    'transition': id_transicao_suporte,
    'fields': campos_customizados
  }

  jira.transition_issue(issue, **transicao_args)

  time.sleep(1)

  # 5. Colocar chamado em andamento e adicionar mensagem
  id_transicao_em_andamento = 00
  jira.transition_issue(issue, id_transicao_em_andamento)

    # Nova mensagem para atestar que está em andamento  
  comentario_andamento = f'O ID da renovação {id_policy_new} está sendo vinculado ao ID renovado {id_policy_old}. Nossa automação está trabalhando nesse processo, informaremos quando for finalizado.'
  jira.add_comment(issue, comentario_andamento)

  # Chamado adicionado ao dicionário
  chamados[id] = issue.key


for chamado in chamados.values():
  print(f'Card {chamado} aberto e em andamento')

# Chamado para nesse estágio. Após criação do merge request, o chamado passa para "Aguardando deploy" e após ser mergeado, é concluído.

###Conexão com o gitlab

In [0]:
### Configura conexão com o Gitlab
#-----------------------------------------------------------------------------


# Configurações
gitlab_url = r"https://gitlab.com"
support_project_id = "project_id" 
gitlab_token = "personal_token"

# Crie uma instância da classe Gitlab
gl = gitlab.Gitlab(gitlab_url, private_token=gitlab_token)

# Autentique-se no GitLab
gl.auth()

# Lendo o projeto em que fazemos update
project = gl.projects.get(support_project_id)

In [0]:
# Informações de data e hora do changeset (Query Head)
agora = datetime.now()
formato = "%Y-%m-%d-%H%M"
data_hora_formatada = agora.strftime(formato)

data_hora_formatada

In [0]:
### Passo a Passo para fase de criação do Merge Request

# 1. Pegar as principais informações do chamado necessárias no processo:
  # Chave do chamado, título do chamado e título tratado
# 2. Criação da branch
# 3. Set nome do arquivo para o commit
# 4. Set o head da query
# 5. Gerador da query
  # Verificar se status id_old = 18 e type_solicitation id_new = 108 (se não, update)
# 6. Criação do Commit
# 7. Criação do Merge Request
# 8. Colocar chamados como 'Aguardando deploy'
# 9. Inserir id_pre_prod na tabela de processados com status = 1

In [0]:
# Criação do Merge Request (famoso PR ou MR)

for id in chamados.keys():
  # 1. Pegando infos principais do chamado para o merge
  singleIssue = jira.issue(chamados[id])
  chave_chamado = singleIssue.key
  titulo_chamado = singleIssue.fields.summary
  branch_name = f.trata_branch_name(titulo_chamado)
  head_query_title = f.trata_head_query_title(titulo_chamado)


  # 2.Criação da branch
  branch_name = f'type/{chave_chamado}-{branch_name}'

  branch = project.branches.create({'branch': branch_name,
                                  'ref': 'master'})
  
  time.sleep(1)

  # 3. Set arquivo
  arquivo_changes = f'file_name/{data_hora_formatada}-{branch_name}.sql'


  # 4. Set query head
  query_head =f'''-- tool formatted sql
-- changeset name lastname:{data_hora_formatada}
-- comment [{chave_chamado}] {head_query_title}'''


  # 5. Gerador da query
    # Infos para a query
  id_policy_new = int(df_pre_prod.loc[df_pre_prod['id'] == id, 'id_policy_new'].iloc[0])
  id_policy_old = int(df_pre_prod.loc[df_pre_prod['id'] == id, 'id_policy_old'].iloc[0])
  email_usuario = df_pre_prod.loc[df_pre_prod['id'] == id, 'email_solicitante'].iloc[0]
  id_user = df_core_user.loc[df_core_user['txt_email'] == email_usuario, 'id'].iloc[0]
  status_id_old = int(df_op_policy.loc[df_op_policy['id'] == id_policy_old, 'Status'].iloc[0])
  type_id_new = int(df_op_policy.loc[df_op_policy['id'] == id_policy_new, 'id_enum_type_solicitation'].iloc[0])

    # Geradores
  if status_id_old == 18 and type_id_new == 108:
    query_vinculo = f.query_renovacao(id_policy_old, id_policy_new, id_user)
  elif not status_id_old == 18 and type_id_new == 108:
    query_vinculo = f.query_renovacao_status(id_policy_old, id_policy_new, id_user, status_id_old)
  elif status_id_old == 18 and not type_id_new == 108:
    query_vinculo = f.query_renovacao_type(id_policy_old, id_policy_new, id_user, type_id_new)
  elif not status_id_old == 18 and not type_id_new == 108:
    query_vinculo = f.query_renovacao_status_type(id_policy_old, id_policy_new, id_user, status_id_old, type_id_new)
  
    # Concatenar query com a head
  query_completa = query_head + '\n\n' + query_vinculo

  time.sleep(1)

  # 6. Fazendo commit dentro da branch
  data = {
      'branch': branch_name,
      'commit_message': f'[{chave_chamado}] {head_query_title}',
      'actions': [
          {
              'action': 'create',
              'file_path': arquivo_changes,
              'content': query_completa,
          }
      ]
  }

  commit = project.commits.create(data)

  time.sleep(1)

  # 7. Merge Request
  mr = project.mergerequests.create({'source_branch': branch_name,
                                   'target_branch': 'master',
                                   'title': f'[{chave_chamado}] {head_query_title}',
                                   'labels': ['bot-renovacao']})
  
  time.sleep(1)
    # Adiciona descrição ao merge
  mr.description = f'Vincular id_new {id_policy_new} ao id_old {id_policy_old}. By: BOT Vínculo de renovação'

    # Atribui o merge para mim
  mr.assignee_id = gl.user.id

  time.sleep(1)
    # Adiciona reviewers ao merge
  reviewers_id = [00000000, 00000000, 00000000, 00000000, 00000000, 00000000, 00000000, 00000000]
    # Reviewers ID's
  mr.reviewer_ids = reviewers_id
  mr.save()

  time.sleep(1)

  # 8. Colocar chamados em aguardando deploy
  issue = chamados[id]
  id_aguardando_deploy = 000

  jira.transition_issue(issue, id_aguardando_deploy)


  # 9. Inserir id_pre_prod na tabela de processados com status = 1 (Aguardando deploy)
  tabela = 'table_name'
  mr_iid = mr.iid

  dados = {
        "id" : id, 
        "status" : 1, # Status 1 = aguardando deploy 
        "chave_chamado" : issue,
        "iid_merge_request" : mr_iid}
  sql_inserir_dados = f"INSERT INTO {tabela} ({', '.join(dados.keys())}) VALUES ({', '.join(['%s']*len(dados))})"
  cursor.execute(sql_inserir_dados, tuple(dados.values()))
  connection.commit()

  time.sleep(1)

# Fechar conexão MySQL
connection.close()